# Gemma 3n Kaggle Test Notebook

This notebook is designed to run on Kaggle with Gemma 3n model for image classification tasks.

In [1]:
import torch
print(f"🧠 Torch CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f'🚀 Using GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}')

🧠 Torch CUDA Available: True
🚀 Using GPU: Tesla P100-PCIE-16GB


In [2]:
# 🧠 KAGGLE Notebook Cell — installs dependencies
!pip install timm --upgrade
!pip install accelerate
!pip install git+https://github.com/huggingface/transformers.git
!pip install kagglehub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nv

In [3]:
# Kaggle-specific imports and setup
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

## ✅ Gemma 3n Image Captioning Test (Patched for P100 GPU)
This notebook uses the `google/gemma-3n-E4B-it` multimodal model from Hugging Face. Make sure your Kaggle runtime is set to **P100 GPU** (or T4 ×2) and `transformers>=4.53.0` is installed.
**Model supports image + text → caption output.**

In [9]:
import kagglehub

GEMMA_PATH = kagglehub.model_download("google/gemma-3n/transformers/gemma-3n-e4b-it")

In [10]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")


# Authenticate with Hugging Face
login(secret_value_0)
print("✅ Hugging Face login successful.")

✅ Hugging Face login successful.


In [13]:
from transformers import AutoProcessor, Gemma3nForConditionalGeneration
from PIL import Image
import torch


# Load model and processor
processor = AutoProcessor.from_pretrained(GEMMA_PATH)
model = Gemma3nForConditionalGeneration.from_pretrained(
    GEMMA_PATH,
    device_map="auto",
    torch_dtype=torch.bfloat16
).eval()


print(f'📦 Model loaded on device: {model.device}')


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

📦 Model loaded on device: cuda:0


In [14]:
# Load test image
img_path = "/kaggle/input/sosimagescnntest/sand_sos.png"
img = Image.open(img_path).convert("RGB")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": img},
            {"type": "text", "text": "Describe this image in detail and look for pattern that matches SOS word."}
        ]
    }
]

# Prepare input
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device)

input_len = inputs["input_ids"].shape[-1]

# Generate
with torch.inference_mode():
    output = model.generate(**inputs, max_new_tokens=128, do_sample=False)
    output = output[0][input_len:]

text = processor.decode(output, skip_special_tokens=True)
print("🖼️ Caption:", text)

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🖼️ Caption: ## Detailed Description of the Image:

The image shows the numbers "2020" deeply imprinted in light brown sand. The numbers are arranged vertically, one below the other, and are clearly formed by someone tracing the digits with their finger or a similar tool. 

The sand is fine-grained and appears slightly damp, allowing the impressions to be well-defined. The texture of the sand is visible, with subtle variations in tone and small grains scattered across the surface. 

The lighting in the image seems to be coming from above, casting soft shadows within the impressions of the numbers, which adds depth and dimension to the
